In [63]:
import gunpowder as gp
import zarr
import os
import numpy as np
import matplotlib.pyplot as plt

In [79]:
ZARR_FOLDER_TRAIN = '../../../../data/cells/raw/zarr2/test/'
gp_batch_size= 1
gp_voxel_shape = [1,1,1]
gp_input_shape= [572, 572]
gp_output_shape = [ 388, 388]

In [97]:
import math
            
def batch_data_generator(input_path,batch_size=12,voxel_shape = [1,1],
                             input_shape= [240, 240],
                             output_shape = [240, 240],
                             without_background = False,
                                 mix_output = False):
    raw0 = gp.ArrayKey('raw0')
    raw1 = gp.ArrayKey('raw1')
    raw2= gp.ArrayKey('raw2')
    raw3 = gp.ArrayKey('raw3')
    gt0 = gp.ArrayKey('ground_truth0')
    gt1 = gp.ArrayKey('ground_truth1')
    gt2 = gp.ArrayKey('ground_truth2')
    files = os.listdir(input_path)
    files = [os.path.join(input_path,f) for f in files ]
#     print(files)
    pipeline =( tuple (
        gp.ZarrSource(
            files[t],  # the zarr container
            {raw0: 'raw0',raw1: 'raw1',raw2: 'raw2',raw3: 'raw3', 
             gt0 : 'ground_truth0',gt1 : 'ground_truth1',gt2 : 'ground_truth2'},  # which dataset to associate to the array key
            {raw0: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             raw1: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             raw2: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             raw3: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             gt0: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             gt1: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape),
             gt2: gp.ArraySpec(interpolatable=True,dtype=np.dtype('float32'),voxel_size=voxel_shape)}  # meta-information
        )
#             +gp.ElasticAugment(   
#                 control_point_spacing=(16,16),
# jitter_sigma=(4.0, 4.0),
#  rotation_interval=(0, math.pi/2)
#             )
#          +gp.SimpleAugment()
        + gp.RandomLocation()
        for t in range(len(files))
    )
#            
              
               + gp.RandomProvider()
#                +gp.Stack(batch_size)
              )
    input_size = gp.Coordinate(input_shape)
#     output_size = gp.Coordinate(output_shape)
    print(input_shape)
    print(input_size)
    request = gp.BatchRequest()
    request.add(raw0,input_size)
#     request.add(raw1,input_size)
#     request.add(raw2,input_size)
#     request.add(raw3,input_size)
#     request.add(gt0,input_size)
#     request.add(gt1,input_size)
#     request.add(gt2,input_size)
    diff = input_shape[1] - output_shape[1]
    diff = int(diff/2)
    max_p = input_shape[1]-diff
    different_shape = diff > 0
    if different_shape:
        print('Difference padding: {}'.format(diff))
    while 1:
        with gp.build(pipeline):
            batch = pipeline.request_batch(request)
            im = np.concatenate((batch[raw0].data, batch[raw1].data,batch[raw2].data,batch[raw3].data), axis=2)
            yield im
            
#             out = batch[gt].data
#             if different_shape:
#                 out = out[:,diff:max_p,diff:max_p,:]
#             if without_background:
#                 out = out[:,:,:,1:4]
#             if mix_output:
#                 out = out.argmax(axis=3).astype(float)
#             yield im, out
            
gen_train_fast = batch_data_generator(ZARR_FOLDER_TRAIN, batch_size=gp_batch_size
                                              ,voxel_shape = gp_voxel_shape,
                                              input_shape= gp_input_shape,
                                              output_shape = gp_output_shape)            

In [98]:
im,mask = next(gen_train_fast)
print(im.shape)
print(mask.shape)
test_im = im[0]
test_mask = mask[0]
np.unique(test_mask)
print(np.unique(test_mask))
print('Img size: {} {} < {} | Mask size: {} {} <{}'.format(test_im.shape,test_im.dtype,test_im.max()
                                                                   ,test_mask.shape,test_mask.dtype,test_mask.max()))
plt.imshow(test_im[:,:,2])
plt.show()
plt.imshow(test_mask[:,:,0])
plt.show()

something went wrong during the setup of the pipeline, calling tear down


[572, 572]
(572, 572)
Difference padding: 92


AssertionError: can only multiply Coordinate of equal dimensions

In [24]:
test_im[:,:,0:3].shape

(572, 572, 3)